# Run Simulations

Given an encoded dataset of targets and queries, run simulations.

In [1]:
# TODO: Consider modifying this to simualate the nucleaseq output score https://github.com/uwmisl/cas9-similarity-search/issues/3

import numpy as np
import pandas as pd

import primo.models

#import cupyck

from tqdm.notebook import tqdm

QUERY = 'callie_janelle'

In [5]:
# hosts = [
#     ("localhost", 2046),
# ]
# client = cupyck.Client(hosts)
# simulator = primo.models.Simulator(client)
simulator = primo.models.Simulator()

In [6]:
#target_seqs = pd.read_hdf('/tf/primo/data/extended_targets/feature_seqs.h5')
target_seqs = pd.read_hdf('/tf/primo/data/targets/feature_seqs.h5')
query_seqs = pd.read_hdf('/tf/primo/data/queries/feature_seqs.h5')

In [8]:
n_unique = len(target_seqs['FeatureSequence'].unique())
n_total = len(target_seqs['FeatureSequence'])
print(n_unique)
print(f"{n_unique} / {n_total} ({float(n_unique) * 100 / n_total:0.1f}%)")

748
748 / 1743042 (0.0%)


In [9]:
pairs = (target_seqs
 .rename(columns={'FeatureSequence':'target_features'})
 .assign(query_features = query_seqs.loc[QUERY].FeatureSequence)
)

In [10]:
pairs

,target_features,query_features
000002b66c9c498e,ATGCTGTGATCACCCATCCC,ATCGCGTGATCAACCACCGC
000002b97e5471a0,AGACTGTGAAAAACCATCCC,ATCGCGTGATCAACCACCGC
000002c707c9895e,ATACTGTGATCAACCATCCC,ATCGCGTGATCAACCACCGC
0000048549557964,GGCCTGTGAAAACCCATCGC,ATCGCGTGATCAACCACCGC
000004f4400f6ec5,CTGCTGTGATCACCCACCCC,ATCGCGTGATCAACCACCGC
...,...,...
ffffd9716fd38279,ATAGTGTGATAACCCATCCC,ATCGCGTGATCAACCACCGC
ffffda81903d6bb7,AGGCTGTGATCACCCACCCC,ATCGCGTGATCAACCACCGC
ffffeac7d2f37489,GGCCTGTGAAAACCCATCCC,ATCGCGTGATCAACCACCGC
ffffebddbb8bba21,ATACTGTGATCAACCATCGC,ATCGCGTGATCAACCACCGC


In [11]:
# 4,000 here is just a memory-management batch size so that each progress chunk reports period of time.
split_size = 4_000
nsplits = len(pairs) / split_size
splits = np.array_split(pairs, nsplits)

In [12]:
result_store = pd.HDFStore(f'/tf/primo/data/simulation/extended_targets/{QUERY}.h5', complevel=9, mode='w')
try:
    for split in tqdm(splits):
        results = simulator.simulate(split)
        result_store.append('df', pd.DataFrame({'duplex_yield': results}, index=split.index))
finally:
    result_store.close()

  0%|          | 0/435 [00:00<?, ?it/s]